In [1]:
# import firedanger module
import os
import xarray as xr
import numpy as np
os.chdir("/home/cshatto/Projects/effis-era5-europe")


In [6]:
fg_0 = "data/era5/processed/E-OBS/fg_0.nc"
hu_0 = "data/era5/processed/E-OBS/hu_0.nc"
rr_0 = "data/era5/processed/E-OBS/rr_0.nc"
tg_0 = "data/era5/processed/E-OBS/tg_0.nc"

fg_0 = xr.open_dataset(fg_0)
hu_0 = xr.open_dataset(hu_0)
rr_0 = xr.open_dataset(rr_0)
tg_0 = xr.open_dataset(tg_0)


In [13]:
rr_0

<xarray.Dataset> Size: 8GB
Dimensions:      (x: 646, y: 386, time: 4018)
Coordinates:
  * x            (x) float64 5kB -24.35 -24.25 -24.15 ... 39.95 40.05 40.15
  * y            (y) float64 3kB 71.15 71.05 70.95 70.85 ... 32.85 32.75 32.65
  * time         (time) datetime64[ns] 32kB 2000-01-01 2000-01-02 ... 2010-12-31
Data variables:
    spatial_ref  int32 4B ...
    rr           (time, y, x) float64 8GB ...

In [14]:
rr_0['x'].values

array([-24.35013951, -24.25013951, -24.15013951, -24.05013951,
       -23.95013951, -23.85013951, -23.75013951, -23.65013951,
       -23.55013951, -23.45013952, -23.35013952, -23.25013952,
       -23.15013952, -23.05013952, -22.95013952, -22.85013952,
       -22.75013952, -22.65013952, -22.55013952, -22.45013952,
       -22.35013952, -22.25013952, -22.15013952, -22.05013952,
       -21.95013952, -21.85013952, -21.75013952, -21.65013952,
       -21.55013952, -21.45013952, -21.35013952, -21.25013952,
       -21.15013952, -21.05013952, -20.95013953, -20.85013953,
       -20.75013953, -20.65013953, -20.55013953, -20.45013953,
       -20.35013953, -20.25013953, -20.15013953, -20.05013953,
       -19.95013953, -19.85013953, -19.75013953, -19.65013953,
       -19.55013953, -19.45013953, -19.35013953, -19.25013953,
       -19.15013953, -19.05013953, -18.95013953, -18.85013953,
       -18.75013953, -18.65013953, -18.55013953, -18.45013954,
       -18.35013954, -18.25013954, -18.15013954, -18.05

In [15]:
# FWI Calculation Functions
def ffmc_calc(temp, rh, wind, precip, ffmc_prev):
    """Calculate Fine Fuel Moisture Code (FFMC)"""
    # Constants
    mo = 147.2 * (101.0 - ffmc_prev) / (59.5 + ffmc_prev)  # Initial moisture content
    if precip > 0.5:
        rf = precip - 0.5
        mo = mo + 42.5 * rf * np.exp(-100.0 / (251.0 - mo)) * (1.0 - np.exp(-6.93 / rf))
        if mo > 250.0:
            mo = 250.0
    ed = 0.942 * (rh ** 0.679) + 11.0 * np.exp((rh - 100.0) / 10.0) + 0.18 * (21.1 - temp) * (1.0 - np.exp(-0.115 * rh))
    if mo > ed:
        k0 = 0.424 * (1.0 - (rh / 100.0) ** 1.7) + 0.0694 * (wind ** 0.5) * (1.0 - (rh / 100.0) ** 8)
        k1 = k0 * (0.581 * np.exp(0.0365 * temp))
        mo = ed + (mo - ed) * np.exp(-k1)
    elif mo < ed:
        ew = 0.618 * (rh ** 0.753) + 10.0 * np.exp((rh - 100.0) / 10.0) + 0.18 * (21.1 - temp) * (1.0 - np.exp(-0.115 * rh))
        if mo < ew:
            k0 = 0.424 * (1.0 - ((100.0 - rh) / 100.0) ** 1.7) + 0.0694 * (wind ** 0.5) * (1.0 - ((100.0 - rh) / 100.0) ** 8)
            k1 = k0 * (0.581 * np.exp(0.0365 * temp))
            mo = ew - (ew - mo) * np.exp(-k1)
    ffmc = 59.5 * (250.0 - mo) / (147.2 + mo)
    return max(0.0, min(ffmc, 101.0))

def dmc_calc(temp, rh, precip, dmc_prev, month):
    """Calculate Duff Moisture Code (DMC)"""
    if temp < -1.1:
        temp = -1.1
    rk = 1.894 * (temp + 1.1) * (100.0 - rh) * 0.000013 * ([6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0][month-1])
    if precip > 1.5:
        pr = 100.0 * (precip - 0.5) / (dmc_prev + 1.0)
        dmc_prev = dmc_prev + pr
    dmc = dmc_prev + rk
    return max(0.0, dmc)

def dc_calc(temp, precip, dc_prev, month):
    """Calculate Drought Code (DC)"""
    if temp < -2.8:
        temp = -2.8
    lf = [0.0, -1.6, -1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4][month-1]
    dc_rate = 0.36 * (temp + 2.8) + lf
    if precip > 2.8:
        pd = precip - 2.8
        dc_prev = dc_prev - 400.0 * np.log(1.0 + 3.937 * pd / (800.0 * np.exp(-dc_prev / 400.0)))
    dc = dc_prev + 0.5 * dc_rate
    return max(0.0, dc)

def isi_calc(ffmc, wind):
    """Calculate Initial Spread Index (ISI)"""
    fwind = np.exp(0.05039 * wind)
    fm = 59.5 * (250.0 - 147.2 * (101.0 - ffmc) / (59.5 + ffmc)) / (147.2 + (101.0 - ffmc))
    isi = 0.208 * fwind * np.exp(-0.05039 * fm)
    return isi

def bui_calc(dmc, dc):
    """Calculate Buildup Index (BUI)"""
    if dmc <= 0.4 * dc:
        bui = 0.8 * dmc * dc / (dmc + 0.4 * dc)
    else:
        bui = dmc - (1.0 - 0.8 * dc / (dmc + 0.4 * dc)) * (0.92 + (0.0114 * dmc) ** 1.7)
    return max(0.0, bui)

def fwi_calc(isi, bui):
    """Calculate Fire Weather Index (FWI)"""
    if bui <= 80.0:
        f = 0.626 * bui ** 0.809 + 2.0
    else:
        f = 1000.0 / (25.0 + 108.64 * np.exp(-0.023 * bui))
    b = 0.1 * isi * f
    if b > 1.0:
        fwi = 2.72 * (0.434 * np.log(b)) ** 0.647
    else:
        fwi = b
    return fwi


# Extract variables
temp = tg_0 #ds['t2m_C']  # Daily max temperature (°C)
rh = hu_0 #ds['RH']    # Daily avg relative humidity (%)
wind = fg_0 #ds['windSpeed'] * 3.6  # Daily mean wind speed (m/s to km/h)
precip = rr_0 #ds['rr']  # Daily precipitation sum (mm)
time = rr_0['time'].values
lat = rr_0['x'].values
lon = rr_0['y'].values

# Initialize output arrays
fwi_output = np.zeros((len(time), len(lat), len(lon)))
ffmc_prev = np.full((len(lat), len(lon)), 85.0)  # Initial FFMC
dmc_prev = np.full((len(lat), len(lon)), 6.0)    # Initial DMC
dc_prev = np.full((len(lat), len(lon)), 15.0)    # Initial DC

# Loop over time and grid
for t in range(len(time)):
    month = int(str(time[t])[5:7])  # Extract month from time (e.g., '2025-03-12' -> 3)
    for i in range(len(lat)):
        for j in range(len(lon)):
            t_val = temp[t, i, j].values
            rh_val = rh[t, i, j].values
            wind_val = wind[t, i, j].values
            precip_val = precip[t, i, j].values

            # Handle NaN
            t_val = 15.0 if np.isnan(t_val) else t_val
            rh_val = 50.0 if np.isnan(rh_val) else rh_val
            wind_val = 0.0 if np.isnan(wind_val) else wind_val
            precip_val = 0.0 if np.isnan(precip_val) else precip_val

            # Calculate indices
            ffmc = ffmc_calc(t_val, rh_val, wind_val, precip_val, ffmc_prev[i, j])
            dmc = dmc_calc(t_val, rh_val, precip_val, dmc_prev[i, j], month)
            dc = dc_calc(t_val, precip_val, dc_prev[i, j], month)
            isi = isi_calc(ffmc, wind_val)
            bui = bui_calc(dmc, dc)
            fwi = fwi_calc(isi, bui)

            # Store result and update previous values
            fwi_output[t, i, j] = fwi
            ffmc_prev[i, j] = ffmc
            dmc_prev[i, j] = dmc
            dc_prev[i, j] = dc

# Create xarray DataArray
fwi_da = xr.DataArray(
    fwi_output,
    coords=[time, lat, lon],
    dims=['time', 'latitude', 'longitude'],
    name='fwi'
)

# Save to NetCDF
fwi_ds = fwi_da.to_dataset()
fwi_ds.to_netcdf("fwi_output_manual.nc")
ds.close()

print("Manual FWI calculation completed and saved to 'fwi_output_manual.nc'.")

KeyError: "No variable named (0, 0, 0). Variables on the dataset include ['x', 'y', 'time', 'spatial_ref', 'tg']\nHint: use a list to select multiple variables, for example `ds[[0, 0, 0]]`"

In [ ]:


# initiate instance and
# read gridded COSMO-1 analysis hourly data from 20180801_00 to 20180814_03 with 0.01° (~1 km) spatial resolution)
fire = firedanger('data/cosmo-1_ana.nc')
print(fire)
# Out[]:     Xarray dataset with 316 time steps.
#            Available fields: TOT_PREC, T_2M, U_10M, V_10M, RELHUM_2M

# preprocessing: select only time at 12 noon
fire.ds = fire.ds.sel(time=datetime.time(12))
# xarray.Dataset (and all its functions) can be accessed with fire.ds

# preprocessing: calculate wind speed
fire.calc_windspeed(u="U_10M", v="V_10M")
# creates new variable "wind"

# Hint: Use fire.set_up(...) to do consistency check and set (automatically or manually) names of dimension ('time', 'latitude', 'longitude')

# calculate Canadian Forest Fire Weather Indices
fire.calc_canadian_fwi(temp="T_2M", precip="TOT_PREC", hum="RELHUM_2M", wind="wind")
print(fire)
# Out[]:     Xarray dataset with 13 time steps.
#            Available fields: TOT_PREC, T_2M, U_10M, V_10M, RELHUM_2M, wind, ffmc, dmc, dc, isi, bui, fwi

# save to disk as netcdf
fire.to_netcdf('data/cosmo-1_daily_fire.nc')

# plot Duff Moisture Code at one timestep
import matplotlib.pyplot as plt
fire.dmc[0].plot(cmap="plasma")
plt.show()